In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import  datasets, transforms

In [2]:
BATCH_SIZE = 512
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
pipeline=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,),(0.3081,))]
)

In [4]:
from torch.utils.data import DataLoader

train_set=datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=pipeline
)

test_set=datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=pipeline
)

train_loader=DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader=DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)


In [5]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
    # 把输出内容保存在 “CNN.txt” 文件中
    with open("../result/RNN.txt", "a") as f:
        f.write("Epoch: %d, Loss: %f\n" % (epoch, loss.item()))
        

In [6]:
def test(model,device,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data,target=data.to(device),target.to(device)
            output=model(data)
            test_loss+=F.cross_entropy(output,target,reduction="sum").item()
            pred=output.max(1,keepdim=True)[1]
            correct+=pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    with open("../result/RNN.txt", "a") as f:
        f.write("Test set: Average loss: %f, Accuracy: %d/%d (%f%%)\n" % (
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)
        ))
    

In [7]:
import time
def train_test(model,device,train_loader,test_loader,optimizer,epoch):
    for epoch in range(1,epoch+1):
        start=time.time()
        train(model,device,train_loader,optimizer,epoch)
        end=time.time()
        with open("../result/RNN.txt", "a") as f:
            f.write("Train Time:%.2f\n" % (end-start))
        start=time.time()
        test(model,device,test_loader)
        end=time.time()
        with open("../result/RNN.txt", "a") as f:
            f.write("Test Time:%.2f\n" % (end-start))

In [8]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.batchnorm = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        # Shape of input is (batch_size,1, 28, 28)
        # converting shape of input to (batch_size, 28, 28)
        # as required by RNN when batch_first is set True
        input = input.reshape(-1, 28, 28)
        output, hidden = self.rnn(input)

        # RNN output shape is (seq_len, batch, input_size)
        # Get last output of RNN
        output = output[:, -1, :]
        output = self.batchnorm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = F.log_softmax(output, dim=1)
        
        return output

In [9]:
def run():
    model = RNN().to(DEVICE)
    print(model)
    # optimizer = optim.Adam(model.parameters())
    # train_test(model, DEVICE, train_loader, test_loader, optimizer,EPOCHS)

In [10]:
if __name__ == '__main__':
    run()

RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (batchnorm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)
